This is a simple reference for loading a geojson layer over a folium map, as well as loading new attributes into the feature set and building a new geojson object from three different ones

In [ ]:
import folium
import json

In [ ]:
print(folium.__version__)

In [ ]:
%ls

WR has an open dataportal where you can find files that describe the different neighbourhoods.
In Waterloo they are 'District Plans'
In Kitchener they are 'Planning Communities'
In Cambridge they are 'Planning Neighbourhoods'

In [ ]:
kfile = 'geojson/Kitchener_Neighbourhoods.geojson'
cfile = 'geojson/Cambridge_Neighbourhoods.geojson'
wfile = 'geojson/Waterloo_Neighbourhoods.geojson'

### load geojson object

In [ ]:
# Kitchener
with open(kfile) as f:
    wr = json.load(f)

In [ ]:
# Cambridge
with open(cfile) as f:
    wr2 = json.load(f)

In [ ]:
# Waterloo
with open(wfile) as f:
    wr3 = json.load(f)

### New Geojson Feature Set

The open data portal provides a way to get access to the shape, KML and geojson files for the three cities in WR. But the ideal would be to have one geojson object that combines all three cities into one file.  To that end we can setup the framework for a new geojson object as follows...

In [ ]:
new_geo = {"type": "FeatureCollection", 
         "name": "Planning_Communities", 
         "crs": { "type": "name", 
                     "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
         "features": []} 

In [ ]:
# we need to append to the features the following structure 
# { "type": "Feature", "properties": { "OBJECTID": 1, "CITY": "Kitchener", "NEIGHBOURHOOD": "Victoria Hills"},
    # geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [x,y],[x2,y2]...[xn,yn]]]]} },

We can explore the strucutre of the geojson object

In [ ]:
len(wr)

In [ ]:
wr.keys()

In [ ]:
print(f"type: {type([wr['features']])}, len: {len(wr['features'])}")

The naming convention in the different geojson objects are all different.   Ideally we want to unify the keys so
there is some level of consistency since they are all going to be compiled into one object. At the same time we can build in a color feature based on the city that polygon/neighbourhood exists in

In [ ]:
gj_objs = [('Kitchener', wr),
           ('Cambridge', wr2),
           ('Waterloo', wr3)]

key_mapping = {'Kitchener': {'NH': "PLANNING_COMMUNITY", 'PCID': 'PLANNINGCOMMUNITYID', 
                             'COMM_LABEL': 'PLANNING_COMMUNITY'},
               'Waterloo': {'NH': 'DISTNAME', 'PCID': 'PLANNINGDI', 'COMM_LABEL': 'DISTNAME'},
               'Cambridge': {'NH': "NHOOD_NAME", 'PCID': "NHOOD_ID", 'COMM_LABEL': "COMMUNITY_NAME"}}

colours = {'Kitchener' : {'fillColor': '#228B22', 'color': '#228B22'},
          'Cambridge' : {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'},
          'Waterloo' : {'fillColor': '#EBBA34', 'color': '#EB7D34'}}

# add features to new geojson object
for obj in gj_objs:
    City, struct = obj
    for feature in struct['features']:
        # insert City as a property
        feature['properties']['CITY'] = City
        # insert common labels for key features
        for k in key_mapping[City].keys():
            feature['properties'][k] = feature['properties'][key_mapping[City][k]]
            
        # insert a colour feature
        feature['properties']['colour'] = colours[City]
        # append feature to new geojson object
        new_geo['features'].append(feature)
    print(f'{City} added')
    print(f"Features are now: {len(new_geo['features'])}")


### Set Geometry Colours

In [ ]:
style1 = {'fillColor': '#228B22', 'color': '#228B22'}
style2 = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}
style3 = {'fillColor': '#EBBA34', 'color': '#EB7D34'}

### create folium map

In [ ]:
kw = {'location': [43.451413, -80.492713], 'zoom_start': 12}

In [ ]:
m = folium.Map(**kw)

In [ ]:
folium.GeoJson(wr,name='Kitchener', 
               style_function= lambda x:style1,
              popup=folium.GeoJsonPopup(fields=['PLANNING_COMMUNITY'])).add_to(m)

In [ ]:
folium.GeoJson(wr2, name='Cambridge', 
               style_function= lambda x:style2,
               popup=folium.GeoJsonPopup(fields=['NHOOD_NAME', 'COMMUNITY_NAME'])).add_to(m)

In [ ]:
folium.GeoJson(wr3, name='Waterloo', 
               style_function= lambda x:style3,
               popup=folium.GeoJsonPopup(fields=['DISTNAME'])).add_to(m)

### make sure the individual geojson objects looks correct

In [ ]:
m

### Load the unified object 

Now we can compare the two maps by building a new map using the unified geojson object and maintain the three colour schemes for the polygons of each city

In [ ]:
def style(feature):
    ''' 
    taken from:
    https://gis.stackexchange.com/questions/394219/folium-draw-polygons-with-distinct-colours
    leverages the fact that each city has a unique colour attribute in the feature space
    '''
    return {'fillColor': feature['properties']['colour']['fillColor'],
            'color': feature['properties']['colour']['color']}

m2 = folium.Map(**kw)
folium.GeoJson(new_geo,name='KW and Cambridge', 
               style_function= style,
               popup=folium.GeoJsonPopup(fields=['CITY','NH','COMM_LABEL'])).add_to(m2)

In [ ]:
m2

Write out the new structure to a file for future use

In [ ]:
with open('Kitchener_Waterloo_Cambridge_NH.geojson', 'w') as out_f:
    json.dump(new_geo, out_f)

...and double check to make sure it works

In [ ]:
with open('Kitchener_Waterloo_Cambridge_NH.geojson') as f:
    wr4 = json.load(f)

In [ ]:
m3 = folium.Map(**kw)
folium.GeoJson(wr4,name='KW and Cambridge', 
               style_function= style,
               popup=folium.GeoJsonPopup(fields=['CITY','NH','COMM_LABEL'])).add_to(m3)
m3